In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.__version__

'0.25.3'

## Read the historical corrected data from Flo
The historical data only has PM2.5 and PM10 values, and we want all of the other data as well. So, we need to process the historical data to make it ready to join to the uncorrected complete data.

In [2]:
# Load corrected data, remove unnecessary column, deal with nesta-2 vs nesta-2-1 issue and parse dates correctly
corrected = pd.read_csv("../Data/BS Sensors/Back data as of Jan 2019/20190307 to 20190823_15min averages_StDenys_6sensors_1.csv")
del corrected['Unnamed: 0']
corrected.site = corrected.site.str.lower()
corrected.site[corrected.site == "nesta-2"] = 'nesta-2-1'
corrected.date = pd.to_datetime(corrected.date, dayfirst=True)

corrected = corrected.dropna(how='all', subset=['pm25_mean', 'pm10_mean'])

corrected['corrected'] = True

/Users/robin/anaconda3/envs/breathingspaces/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [3]:
# Time range of corrected data
corrected.date.min(), corrected.date.max()

(Timestamp('2019-03-07 00:00:00'), Timestamp('2019-08-23 05:45:00'))

In [4]:
# Sites in corrected data
corrected.site.value_counts().index

Index(['nesta-4', 'nesta-5', 'nesta-7', 'nesta-6', 'nesta-2-1', 'nesta-1',
       'nesta-8'],
      dtype='object')

## Read the full past data set from Flo
This contains all of the past data available at the time of export. None of this has been corrected

In [5]:
# Load all past data, deal with nesta-2 vs nesta-2-1 issue
all_past_data = pd.read_csv('../Data/BS Sensors/Back data as of Jan 2019/aq.csv', names=['location', 'timestamp',
                                                           'temperature', 'humidity',
                                                           'pm25', 'pm10', 'count',
                                                           'pm_sensor_count', 'temphum_sensor_count', 'unknown'])
all_past_data.location[all_past_data.location == "nesta-2"] = 'nesta-2-1'
all_past_data['timestamp'] = pd.to_datetime(all_past_data.timestamp)

all_past_data.loc[all_past_data.temphum_sensor_count == 0, 'temperature'] = np.nan
all_past_data.loc[all_past_data.temphum_sensor_count == 0, 'humidity'] = np.nan

/Users/robin/anaconda3/envs/breathingspaces/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/robin/anaconda3/envs/breathingspaces/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
# Time range of full past data
all_past_data.timestamp.min(), all_past_data.timestamp.max()

(Timestamp('2019-03-15 16:56:27.243513'),
 Timestamp('2020-01-14 10:45:12.427561'))

In [7]:
# Sensors included in full past data
all_past_data.location.value_counts()

nesta-6          25174
nesta-7          21687
nesta-5          21489
nesta-4          20572
aurn-3           19441
nesta-2-1        19304
aurn-4           18570
aurn-2           17727
b2-new-forest    16084
nesta-1          14686
aurn-1           13847
nesta-8           8555
nesta-11          6363
b1-lanchester     6172
nesta-9           5916
nesta-12          5223
nesta-13          1029
nesta-10           672
Name: location, dtype: int64

## Do the merge
We can't just merge the data as the corrected data has times rounded to the nearest 15 minutes, whereas the full past data has the full times. We also can't just set an index and resample because we have multiple measurements for each time period (one for each sensor). So we need to split the data into separate datasets per sensor, and then do the merge for each of those separately.

In [8]:
# For each site that we have corrected data for
# Extract just that site from the whole past data and resample to 15M intervals to match with the perfect 15min
# timestamps that come from the corrected data
# Extract just that site from the corrected data too
# Then merge the two

merged_outputs = []

for sensor in corrected.site.value_counts().index:
    all_one_sensor = all_past_data[all_past_data.location == sensor].copy()
    all_one_sensor.index = pd.to_datetime(all_one_sensor.timestamp)
    all_one_sensor_resampled = all_one_sensor.resample('15Min').first()

    corrected_one_sensor = corrected[corrected.site == sensor].copy()
    corrected_one_sensor.index = pd.to_datetime(corrected_one_sensor.date, dayfirst=True)

    merged = pd.merge(all_one_sensor_resampled, corrected_one_sensor, how='left', left_index=True, right_index=True)
    # There will be some fully blank rows in here because of extra blank rows created by resampling
    # things to be a 15Min interval. We can get rid of them now (but not before now, as we're doing a left join
    # and that would mean values that are just in the corrected data would be left out)
    merged = merged.dropna(how='all')
    merged_outputs.append(merged.reset_index(drop=True))
    
merged = pd.concat(merged_outputs)

## Add in the data for sensors that weren't in the corrected data
Some of the sensors were created after the correction was done, so their data is not in the merged data above. We need to just concatenate that into the dataframe now.

In [9]:
# Newer sensors have no corrected data
newer_sensors = set(all_past_data.location.value_counts().index).difference(set(merged.site.value_counts().index))

In [10]:
newer_sensors

{'aurn-1',
 'aurn-2',
 'aurn-3',
 'aurn-4',
 'b1-lanchester',
 'b2-new-forest',
 'nesta-10',
 'nesta-11',
 'nesta-12',
 'nesta-13',
 'nesta-9'}

In [11]:
# Important to reset the indices here so they don't get used in the merge or brought into the results
final_result = pd.concat([merged.reset_index(drop=True),
                          all_past_data[all_past_data.location.isin(newer_sensors)].reset_index(drop=True)], sort=False)
# For efficiency, make the index a RangeIndex rather than an Int64Index
final_result = final_result.reset_index(drop=True)

## Combine the columns in the right order
We now have a data frame containing two columns for various important bits of data (sensor_id, pm25, pm10 etc). We need to take the corrected column and then fill in the NaN gaps with the uncorrected data. We also fill NaN values in the 'corrected' column with False values (so it is a true boolean), and drop the unused columns once we've done the merge.

In [12]:
final_result.columns

Index(['location', 'timestamp', 'temperature', 'humidity', 'pm25', 'pm10',
       'count', 'pm_sensor_count', 'temphum_sensor_count', 'unknown', 'site',
       'date', 'pm25_mean', 'pm10_mean', 'pm_count', 'corrected'],
      dtype='object')

In [13]:
final_result['site'] = final_result.site.fillna(final_result.location)
final_result['timestamp'] = final_result.date.fillna(final_result.timestamp)
final_result['pm25_mean'] = final_result.pm25_mean.fillna(final_result.pm25)
final_result['pm10_mean'] = final_result.pm10_mean.fillna(final_result.pm10)
final_result['pm_count'] = final_result.pm_count.fillna(final_result.pm_sensor_count)
final_result['corrected'] = final_result.corrected.fillna(False)

final_result = final_result.drop(columns=['location', 'date', 'pm25', 'pm10', 'pm_sensor_count', 'unknown'])

In [14]:
# Add useful field for later - 'display' = True, so we can eventually use it for filtering for display if necessary
final_result['display'] = True

## Subset to just data that isn't already in InfluxDB
The automatic MQTT import to InfluxDB means that some data is already in there. The earliest data points available in InfluxDB are from the 12th Dec 2019 at 10:00, so we just want data from before that

In [15]:
final_result_subset = final_result[final_result.timestamp < pd.to_datetime('2019-12-12 10:00')]

In [16]:
final_result_subset.timestamp.min(), final_result_subset.timestamp.max()

(Timestamp('2019-03-15 16:56:27.243513'),
 Timestamp('2019-12-12 09:45:11.053821'))

## Remove data from times when sensors were known to be malfunctioning
Once we've removed this data, we put it in a separate data frame so we can import it later once it has been cleaned.

In [17]:
corrected_data_end_time = '2019-08-23 05:45:00'

In [18]:
# This is the data from the 'Generate Grafana Expressions' notebook in the BS/Code directory, with the dates
# manually altered so that we're not excluding data during the corrected period (as all this data is corrected and ok)
data = {
        'nesta-7': [[corrected_data_end_time, '2019-09-03']],
        'nesta-9': [['2019-01-01', '2019-10-15'], ['2019-11-14 08:00:00', '2019-11-14 12:00:00']], # Not in corrected data, so remove all
        'nesta-4': [['2019-10-15', '2019-10-31 13:00:00']],
        'nesta-1': [[corrected_data_end_time, '2019-11-13 13:00:00']],
        'nesta-11': [['2019-11-14 08:00:00', '2019-11-14 12:00:00']],
       }

In [19]:
extracted_data = []

for sensor_id, periods in data.items():
    just_this_sensor = final_result_subset[final_result_subset.site == sensor_id]
    for period in periods:
        period_start = pd.to_datetime(period[0])
        period_end = pd.to_datetime(period[1])
        data_in_period = just_this_sensor[(just_this_sensor.timestamp > period_start) & (just_this_sensor.timestamp < period_end)]
        extracted_data.append(data_in_period)
        final_result_subset = final_result_subset.drop(data_in_period.index, axis=0)

In [20]:
masked_data = pd.concat(extracted_data)

In [21]:
masked_data.to_csv('../Data/BS Sensors/MaskedData_NotImportedToInflux_KnownIncorrect_Extracted2019-01-22.csv', index=False)

## Convert into the right format for importing into InfluxDB
We need the CSV in the right format and the columns etc to match what we've already got in Influx for the live MQTT data.

Useful docs:
 - https://www.influxdata.com/blog/how-to-write-points-from-csv-to-influxdb/
 - https://github.com/influxdata/telegraf/tree/master/plugins/parsers/csv
 - https://docs.influxdata.com/influxdb/v1.7/write_protocols/line_protocol_tutorial/

In [22]:
final_result_subset.columns

Index(['timestamp', 'temperature', 'humidity', 'count', 'temphum_sensor_count',
       'site', 'pm25_mean', 'pm10_mean', 'pm_count', 'corrected', 'display'],
      dtype='object')

In [23]:
final_result_subset.columns = ['timestamp', 'p_temperature', 'p_humidity', 'p_count', 'p_temphum_sensor_count',
       'dev_id', 'p_pm25', 'p_pm10', 'p_pm_sensor_count', 'p_corrected', 'p_display']

In [24]:
final_result_subset['timestamp'] = final_result_subset.timestamp.astype(int).astype(str)

In [25]:
final_result_subset['p_count'] = final_result_subset['p_count'].astype('Int16')
final_result_subset['p_temphum_sensor_count'] = final_result_subset['p_temphum_sensor_count'].astype('Int16')
final_result_subset['p_count'] = final_result_subset['p_count'].astype('Int16')
final_result_subset['p_pm_sensor_count'] = final_result_subset['p_pm_sensor_count'].astype('Int16')

In [26]:
final_result_subset['p_pm25'] = final_result_subset['p_pm25'].round(0).astype('Int16')
final_result_subset['p_pm10'] = final_result_subset['p_pm10'].round(0).astype('Int16')

In [27]:
len(final_result_subset)

213899

## Write outputs to CSVs

In [28]:
final_result_subset.to_csv('../Data/BS Sensors/DataForImportToInflux_2019-01-22.csv', index=False, na_rep="null")

In [29]:
sample = final_result_subset.sample(50)

In [30]:
sample.to_csv('../Data/BS Sensors/DataForImportToInflux_2019-01-22_Sample.csv', index=False, na_rep="null")

In [31]:
sample

,timestamp,p_temperature,p_humidity,p_count,p_temphum_sensor_count,dev_id,p_pm25,p_pm10,p_pm_sensor_count,p_corrected,p_display
105878,1563352200000000000,NaN,NaN,NaN,NaN,nesta-2-1,5,6,6,True,True
198914,1568974507723571000,12.0,10.0,34,3,aurn-3,5,7,10,False,True
167313,1561874415114659000,10.0,30.0,35,3,aurn-1,5,6,10,False,True
73,1552734900000000000,16.0,72.0,61,1,nesta-4,5,10,6,True,True
19541,1575147606464074000,8.0,65.0,112,1,nesta-4,8,8,5,False,True
189906,1566282609233591000,12.0,18.0,47,3,aurn-3,6,8,10,False,True
12315,1563752700000000000,25.0,70.0,82,1,nesta-4,6,6,6,True,True
14139,1565394300000000000,29.0,40.0,94,2,nesta-4,10,20,6,True,True
189732,1566223208458093000,13.0,14.0,28,3,aurn-4,5,6,10,False,True
226320,1575051307455686000,7.0,100.0,4,1,b2-new-forest,5,5,4,False,True


## Investigations on missing data
For getting info to write email to Flo asking questions

In [32]:
n5 = corrected[corrected.site == 'nesta-5']

In [33]:
n5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13376 entries, 59475 to 75722
Data columns (total 6 columns):
site         13376 non-null object
date         13376 non-null datetime64[ns]
pm25_mean    13376 non-null float64
pm10_mean    13376 non-null float64
pm_count     13376 non-null int64
corrected    13376 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 640.1+ KB


In [34]:
sum(corrected.pm25_mean.isna() == True)

0

In [35]:
corrected[corrected.pm25_mean.isna()].site.value_counts()

Series([], Name: site, dtype: int64)

In [36]:
n5p = all_past_data[all_past_data.location == 'nesta-5']

In [37]:
n5p[n5p.timestamp > '2019-11-19 08:00']

,location,timestamp,temperature,humidity,pm25,pm10,count,pm_sensor_count,temphum_sensor_count,unknown
171007,nesta-5,2019-11-19 08:00:08.353107,NaN,NaN,22,19,292,6,0,False
171019,nesta-5,2019-11-19 08:15:07.566702,NaN,NaN,23,21,320,6,0,False
171031,nesta-5,2019-11-19 08:30:07.218356,NaN,NaN,20,18,266,6,0,False
171047,nesta-5,2019-11-19 08:45:07.906991,NaN,NaN,19,17,248,6,0,False
171058,nesta-5,2019-11-19 09:00:08.038612,NaN,NaN,20,18,260,6,0,False
...,...,...,...,...,...,...,...,...,...,...
242455,nesta-5,2020-01-14 09:45:07.877982,NaN,NaN,6,12,72,6,0,False
242466,nesta-5,2020-01-14 10:00:07.823645,NaN,NaN,6,10,67,6,0,False
242478,nesta-5,2020-01-14 10:15:07.436590,NaN,NaN,6,11,71,6,0,False
242488,nesta-5,2020-01-14 10:30:06.888683,NaN,NaN,6,11,69,6,0,False


In [38]:
all_past_data[(all_past_data.temperature == 0) & (all_past_data.location == 'nesta-4')]

,location,timestamp,temperature,humidity,pm25,pm10,count,pm_sensor_count,temphum_sensor_count,unknown
